In [ ]:
import sys
import os
import numpy as np
import tensorflow as tf
import pennylane as qml
import pandas as pd
from matplotlib import pyplot as plt

from quantum_neural_network import qnode_entangling, qnode_strong_entangling
from stat_functions import quantitative_analysis, get_mean_left_right_error_interval, verify_distribution_wilcoxtest
from experiments_main import carregar_tabela
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## Plot Functions

In [ ]:
def plot_history(history, n_layers):
    plt.figure(figsize=(14,5), dpi=320, facecolor='w', edgecolor='k')
    plt.title(f"Loss for depth {n_layers}")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.plot(history.history['loss'], label="Loss/Epoch")
    plt.plot(history.history['val_loss'], label="Val Loss/Epoch")
    plt.xticks(range(0, len(history.history['loss'])+1, 5))
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
def plot_prediction_versus_observed(n_layers, y_test, y_pred, mean_error_normal):
    for i in range(y_test.shape[1]):
        plt.figure(figsize=(20,5), dpi=320, facecolor='w', edgecolor='k')
        plt.title(f"Wind Speed Forecast for {i+1} hours ahead for {n_layers} layers")
        plt.xlabel("Samples")
        plt.ylabel("Wind Speed (m/s)")
        plt.plot(y_pred[:,i], label="Prediction", color='blue')
        plt.fill_between(range(y_pred.shape[0]), y_pred[:,i]-mean_error_normal[0,i], y_pred[:,i]+mean_error_normal[0,i], color='blue', alpha=0.05)
        plt.plot(y_test[:,i], label="Original", color='orange')
        plt.legend()
        plt.show()


## Importing Data

In [ ]:
prev = 1
data_folder = "data"
city = "mucuri"
height = "150"

train_file = data_folder+"/"+city+"/"+height+"/"+"train"+height+"_"+city+".txt"
X_all,y_all = carregar_tabela(train_file, prev)

test_file = data_folder+"/"+city+"/"+height+"/"+"prev"+height+"_"+city+".txt"
X_test,y_test = carregar_tabela(test_file,prev)

n_features = X_all.shape[1]
n_instances = X_all.shape[0]
print(f"There are {n_features} features and {n_instances} instances in Train set")
print(f"There are {X_test.shape[1]} features and {X_test.shape[0]} instances in Test set")

In [ ]:
X_all.head()

In [ ]:
X_test.head()

## Scaling Data

In [ ]:
scaler_x = MinMaxScaler(feature_range=(-1, 1))
#scaler_x = StandardScaler()
X_all_scaled = scaler_x.fit_transform(X_all)

X_test_scaled = scaler_x.transform(X_test)

In [ ]:
print(X_all_scaled[0:5])

In [ ]:
print(X_test_scaled[0:5])

## Spliting Train, Validation and Test sets

In [ ]:
train_ratio = 0.8
X_train, X_val, y_train, y_val = train_test_split(X_all_scaled, y_all, test_size=1 - train_ratio)

print("Len(Train):",len(X_train))
print("Len(Val):"  ,len(X_val))
print("Len(Test):" ,len(X_test_scaled))

## Quantum Neural Network

In [ ]:
n_qubits = n_features
n_layers = 2
print(f"Circuit size: {n_qubits} qubits")

In [ ]:
def create_quantum_model(n_layers, n_qubits, strong_entangling=False):
    print(f"Training with depth {n_layers}")
    weight_shapes = {"weights": (n_layers,n_qubits,3)}

    if strong_entangling:
        q_layer = qml.qnn.KerasLayer(qnode_strong_entangling, weight_shapes, output_dim=n_qubits)
    else:
        q_layer = qml.qnn.KerasLayer(qnode_entangling, weight_shapes, output_dim=n_qubits)
    activation=tf.keras.layers.Activation(tf.keras.activations.relu)
    output_layer = tf.keras.layers.Dense(prev,kernel_initializer='normal')

    optimizer = tf.keras.optimizers.Adamax(learning_rate=0.1)

    model = tf.keras.models.Sequential([q_layer, activation, output_layer])
    model.compile(loss=['mse'], optimizer=optimizer, metrics=['mae'])

    return model


In [ ]:
model = create_quantum_model(n_layers, n_qubits)
input_shape = (n_qubits,)
model.build(input_shape)
print(model.summary())

In [ ]:
es=EarlyStopping(monitor='val_loss', min_delta=0, patience=6, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
re=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=0, mode='min', min_lr=0.00001)
history_model = model.fit(X_train, y_train
                        , epochs=50, batch_size=32
                        , verbose=1
                        , validation_data=(X_val, y_val))

## Plotting Loss

In [ ]:
plot_history(history_model, n_layers)

## Prediction

In [ ]:
y_pred = model.predict(X_test_scaled,verbose=1)
mean_predictions, mean_error_normal, mean_error_left_normal, mean_error_right_normal = get_mean_left_right_error_interval(model, scaler_x, X_val, y_val, y_test, y_pred)
plot_prediction_versus_observed(n_layers, y_test, y_pred, mean_error_normal)


## Statistical Analysis

In [ ]:
all_analysis = quantitative_analysis(y_test, [y_pred])
all_analysis

In [ ]:
verify_distribution_wilcoxtest(y_test[:,0],y_pred[:,0], 0.05)

## Creating model with strong entangling and searching for statistical difference

In [ ]:
strong_model = create_quantum_model(n_layers, n_qubits, strong_entangling=True)
input_shape = (n_qubits,)
strong_model.build(input_shape)
strong_model.summary()

In [ ]:
strong_history_model = strong_model.fit(X_train, y_train
                        , epochs=50, batch_size=32
                        , verbose=1
                        , validation_data=(X_val, y_val))

In [ ]:
plot_history(strong_history_model, n_layers)

In [ ]:
strong_y_pred = strong_model.predict(X_test_scaled,verbose=1)
mean_predictions, mean_error_normal, mean_error_left_normal, mean_error_right_normal = get_mean_left_right_error_interval(model, scaler_x, X_val, y_val, y_test, strong_y_pred)
plot_prediction_versus_observed(n_layers, y_test, strong_y_pred, mean_error_normal)


In [ ]:
strong_analysis = quantitative_analysis(y_test, [strong_y_pred])
strong_analysis

In [ ]:
verify_distribution_wilcoxtest(y_pred[:,0],strong_y_pred[:,0], 0.05)

## Creating deeper model and searching for statistical difference

In [ ]:
n_layers = 6
model2 = create_quantum_model(n_layers, n_qubits, strong_entangling=False)
input_shape = (n_qubits,)
model2.build(input_shape)
model2.summary()

In [ ]:
history_model2 = model2.fit(X_train, y_train
                        , epochs=50, batch_size=32
                        , verbose=1
                        , validation_data=(X_val, y_val))

In [ ]:
plot_history(history_model2, n_layers)

In [ ]:
y_pred2 = model2.predict(X_test_scaled,verbose=1)
mean_predictions, mean_error_normal, mean_error_left_normal, mean_error_right_normal = get_mean_left_right_error_interval(model, scaler_x, X_val, y_val, y_test, y_pred2)
plot_prediction_versus_observed(n_layers, y_test, y_pred2, mean_error_normal)


In [ ]:
all_analysis2 = quantitative_analysis(y_test, [y_pred2])
all_analysis2

In [ ]:
verify_distribution_wilcoxtest(y_pred[:,0],y_pred2[:,0], 0.05)